In [1]:
library(tableone)
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.2     ✔ dplyr   0.7.4
✔ tidyr   0.8.0     ✔ stringr 1.3.0
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
data_dir <- file.path("..", "data")

In [3]:
full_data <- readRDS(file.path(data_dir, "full_data_ps.rds"))
head(full_data)

icustay_id,hadm_id,subject_id,first_careunit,intime,outtime,angus,age,icu_order,echo_time,⋯,lab_pco2_flag,lab_creatinine_flag,lab_potassium_flag,lab_troponin_flag,lab_po2_flag,lab_lactate_flag,echo_int,mort_28_day_int,ps,ps_weight
228416,159895,5491,MICU,2151-06-29 01:18:27,2151-07-27 16:00:56,1,76.58225,1,2151-06-30,⋯,1,1,1,0,1,1,1,0,0.5525232,1.809879
278148,169700,49395,MICU,2197-01-09 00:45:42,2197-01-10 06:46:29,1,82.72606,1,NA,⋯,0,1,1,0,0,0,0,1,0.2804613,1.389779
274174,103722,14855,MICU,2181-09-08 23:43:13,2181-09-26 16:52:02,1,61.84390,1,2181-09-11,⋯,1,1,1,0,1,1,1,0,0.3012020,3.320031
214954,170515,54642,MICU,2176-03-11 04:58:23,2176-03-15 18:00:48,1,63.86507,1,2176-03-11,⋯,1,1,1,1,1,1,1,0,0.7982833,1.252688
222457,183493,96815,MICU,2105-06-04 20:57:55,2105-06-06 20:01:51,1,62.84018,1,NA,⋯,0,1,1,0,0,0,0,0,0.2244447,1.289399
264413,134244,81436,MICU,2133-03-27 16:31:37,2133-04-04 16:15:28,1,62.60148,1,2133-03-31,⋯,0,1,1,0,0,1,1,0,0.3320492,3.011602


In [4]:
features <- c("age", "gender", "first_careunit", "weight",
              "saps", "sofa", "elix_score", "vent", "vaso", "sedative",
              "icd_chf", "icd_afib", "icd_renal", "icd_liver",
              "icd_copd", "icd_cad", "icd_stroke", "icd_malignancy",
              "icu_adm_weekday", "icu_adm_hour",
              "vs_map_first", "vs_heart_rate_first", "vs_temp_first", "vs_cvp_first",
              "lab_wbc_first", "lab_hemoglobin_first", "lab_platelet_first",
              "lab_sodium_first", "lab_potassium_first", "lab_bicarbonate_first",
              "lab_chloride_first", "lab_bun_first", "lab_lactate_first",
              "lab_creatinine_first", "lab_ph_first", "lab_po2_first", "lab_pco2_first",
              "lab_bnp_flag", "lab_troponin_flag", "lab_creatinine_kinase_flag")

In [5]:
missing <- full_data %>%
select(features) %>%
summarise_all(funs(sum(is.na(.)) / length(.) * 100)) %>%
gather("cov", "na") %>%
mutate(na = paste0(round(na, 1), "%"))

missing

cov,na
age,0%
gender,0%
first_careunit,0%
weight,9.4%
saps,0%
sofa,0%
elix_score,0%
vent,0%
vaso,0%
sedative,0%


In [6]:
tab <- CreateTableOne(vars = features,
                      strata = "echo",
                      data = full_data,
                      argsNormal = list(var.equal = FALSE))
capture.output(tab_df <- tab %>%
               print(smd = TRUE) %>%
               as.data.frame(stringsAsFactors = FALSE) %>%
               select(-test)) %>% invisible

In [7]:
tab_df

,0,1,p,SMD
n,3099,3262,,
age (mean (sd)),66.69 (17.21),65.74 (16.55),0.026,0.056
gender = F (%),1575 (50.8),1558 (47.8),0.016,0.061
first_careunit = MICU (%),2404 (77.6),2593 (79.5),0.067,0.047
weight (mean (sd)),78.56 (23.58),83.17 (26.87),<0.001,0.182
saps (mean (sd)),19.63 (5.79),20.76 (5.44),<0.001,0.202
sofa (mean (sd)),5.31 (3.62),6.33 (3.79),<0.001,0.277
elix_score (mean (sd)),8.51 (7.45),10.07 (7.67),<0.001,0.207
vent = 1 (%),1459 (47.1),1901 (58.3),<0.001,0.226
vaso = 1 (%),839 (27.1),1228 (37.6),<0.001,0.227


In [8]:
ipw_svydesign <- readRDS(file.path(data_dir, "ipw_svydesign.rds"))

In [9]:
tab_wtd <- svyCreateTableOne(vars = features,
                             strata = "echo",
                             data = ipw_svydesign)

In [10]:
capture.output(tab_wtd_df <- tab_wtd %>%
               print %>%
               as.data.frame(stringsAsFactors = FALSE) %>%
               select(-test)) %>% invisible

In [11]:
tab_wtd_df

,0,1,p
n,5739.48,5842.36,
age (mean (sd)),65.93 (16.29),65.15 (16.73),0.076
gender = F (%),2824.2 (49.2),2824.0 (48.3),0.523
first_careunit = MICU (%),4473.6 (77.9),4641.0 (79.4),0.178
weight (mean (sd)),82.31 (23.50),83.50 (25.25),0.163
saps (mean (sd)),23.13 (5.60),23.41 (5.18),0.134
sofa (mean (sd)),8.25 (4.25),8.52 (3.79),0.038
elix_score (mean (sd)),10.19 (7.45),10.02 (7.81),0.114
vent = 1 (%),2907.8 (50.7),3139.0 (53.7),0.025
vaso = 1 (%),1759.7 (30.7),1965.5 (33.6),0.020


In [12]:
ps_matches_df <- data.table::fread(file.path(data_dir, "ps_matches_df.csv"), data.table = FALSE)

head(ps_matches_df)

icustay_id,group,match
262436,ctrl,1
244460,ctrl,2
298177,ctrl,3
290165,ctrl,4
278397,ctrl,5
212022,ctrl,6


In [13]:
ps_df <- ps_matches_df %>%
left_join(full_data, by = "icustay_id") %>%
arrange(desc(group), match)

head(ps_df)

icustay_id,group,match,hadm_id,subject_id,first_careunit,intime,outtime,angus,age,⋯,lab_pco2_flag,lab_creatinine_flag,lab_potassium_flag,lab_troponin_flag,lab_po2_flag,lab_lactate_flag,echo_int,mort_28_day_int,ps,ps_weight
228416,trtd,1,159895,5491,MICU,2151-06-29 01:18:27,2151-07-27 16:00:56,1,76.58225,⋯,1,1,1,0,1,1,1,0,0.5525232,1.809879
274174,trtd,2,103722,14855,MICU,2181-09-08 23:43:13,2181-09-26 16:52:02,1,61.84390,⋯,1,1,1,0,1,1,1,0,0.3012020,3.320031
214954,trtd,3,170515,54642,MICU,2176-03-11 04:58:23,2176-03-15 18:00:48,1,63.86507,⋯,1,1,1,1,1,1,1,0,0.7982833,1.252688
264413,trtd,4,134244,81436,MICU,2133-03-27 16:31:37,2133-04-04 16:15:28,1,62.60148,⋯,0,1,1,0,0,1,1,0,0.3320492,3.011602
214106,trtd,5,123152,74869,MICU,2150-09-05 17:50:32,2150-09-12 21:40:59,1,74.13371,⋯,1,1,1,1,1,1,1,1,0.7544158,1.325529
266275,trtd,6,107556,10363,MICU,2120-06-05 11:01:07,2120-06-08 19:43:19,1,45.96256,⋯,0,1,1,0,0,0,1,0,0.2709719,3.690419


In [14]:
tab_ps_pval_df <- features %>% map(function(feature) {
    df <- ps_df %>%
    select(!! as.name(feature), match, echo) %>%
    filter(!is.na(!! as.name(feature)))
    matched <- (df %>% filter(echo == 0) %>% rename(ctrl = (!! as.name(feature))) %>% select(ctrl, match)) %>%
    inner_join(df %>% filter(echo == 1) %>% rename(trtd = (!! as.name(feature))) %>% select(trtd, match),
              by = "match") %>%
    filter(!is.na(ctrl), !is.na(trtd))
    
    if (is.factor(ps_df %>% pull(feature))) {
        tab <- table(matched$trtd, matched$ctrl, dnn = c("Echo", "Non-Echo"))
        if (nlevels(ps_df %>% pull(feature)) > 2) {
            tst <- coin::mh_test(tab)
            p.val <- coin::pvalue(tst)
        } else {
            tst <- mcnemar.test(tab)
            p.val <- tst$p.val
        }
        tst <- mcnemar.test(tab)
    } else {
        tst <- t.test(matched$trtd, matched$ctrl, paired = TRUE)
        p.val <- tst$p.val
    }
    
    data.frame(cov = feature, pval = p.val, stringsAsFactors = FALSE)
}) %>%
data.table::rbindlist() %>%
as.data.frame(stringsAsFactors = FALSE) %>%
mutate(pval = case_when(
    pval < 0.001 ~ "0.001",
    TRUE ~ as.character(round(pval, 3))
)) %>%
rbind(data.frame(cov = "n", pval = "", stringsAsFactors = FALSE), .) %>%
select(cov, pval)

tab_ps_pval_df

cov,pval
n,
age,0.209
gender,0.572
first_careunit,0.734
weight,0.06
saps,0.008
sofa,0.001
elix_score,0.005
vent,0.108
vaso,0.375


In [15]:
tab_ps <- CreateTableOne(vars = features,
                         strata = "echo",
                         data = ps_df)
tab_ps

                                    Stratified by echo
                                     0               1               p     
  n                                    1626            1626                
  age (mean (sd))                     66.58 (16.54)   65.84 (17.07)   0.212
  gender = F (%)                        780 (48.0)      797 (49.0)    0.575
  first_careunit = MICU (%)            1274 (78.4)     1283 (78.9)    0.732
  weight (mean (sd))                  81.35 (24.69)   79.50 (23.11)   0.035
  saps (mean (sd))                    20.29 (5.42)    19.80 (5.17)    0.008
  sofa (mean (sd))                     5.99 (3.74)     5.55 (3.50)   <0.001
  elix_score (mean (sd))               9.54 (7.67)     8.82 (7.52)    0.006
  vent = 1 (%)                          851 (52.3)      806 (49.6)    0.123
  vaso = 1 (%)                          531 (32.7)      507 (31.2)    0.387
  sedative = 1 (%)                      713 (43.8)      696 (42.8)    0.571
  icd_chf = 1 (%)                

In [16]:
capture.output(tab_ps_df <- tab_ps %>%
               print(smd = TRUE) %>%
               as.data.frame(stringsAsFactors = FALSE) %>%
               select(-test)) %>% invisible

In [17]:
tab_ps_df

,0,1,p,SMD
n,1626,1626,,
age (mean (sd)),66.58 (16.54),65.84 (17.07),0.212,0.044
gender = F (%),780 (48.0),797 (49.0),0.575,0.021
first_careunit = MICU (%),1274 (78.4),1283 (78.9),0.732,0.014
weight (mean (sd)),81.35 (24.69),79.50 (23.11),0.035,0.077
saps (mean (sd)),20.29 (5.42),19.80 (5.17),0.008,0.092
sofa (mean (sd)),5.99 (3.74),5.55 (3.50),<0.001,0.123
elix_score (mean (sd)),9.54 (7.67),8.82 (7.52),0.006,0.096
vent = 1 (%),851 (52.3),806 (49.6),0.123,0.055
vaso = 1 (%),531 (32.7),507 (31.2),0.387,0.032


In [18]:
wtd_pval <- tab_wtd_df %>%
setNames(c("trtd", "ctrl", "pval")) %>%
mutate(cov = rownames(.)) %>%
mutate_all(funs(str_replace_all(., "\\s+$|^\\s+", ""))) %>%
mutate(cov = strsplit(cov, " ") %>% map_chr(first)) %>%
filter(!is.null(pval) & !is.na(pval) & pval != "") %>%
mutate(pval = str_replace_all(pval, "<", "")) %>%
mutate(pval = as.numeric(pval)) %>%
select(cov, pval)

wtd_pval

cov,pval
age,0.076
gender,0.523
first_careunit,0.178
weight,0.163
saps,0.134
sofa,0.038
elix_score,0.114
vent,0.025
vaso,0.020
sedative,0.030


In [19]:
tab_fmt <- tab_df %>%
setNames(c("ctrl", "trtd", "pval", "smd")) %>%
mutate(cov = rownames(.)) %>%
mutate(ctrl = ifelse((grepl("%", cov) & grepl("=", cov)) | grepl("^\\s+", cov),
                     str_extract(ctrl, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     ctrl)) %>%
mutate(trtd = ifelse((grepl("%", cov) & grepl("=", cov)) | grepl("^\\s+", cov),
                     str_extract(trtd, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     trtd)) %>%
select(cov, ctrl, trtd, pval, smd)

tab_fmt

cov,ctrl,trtd,pval,smd
n,3099,3262,,
age (mean (sd)),66.69 (17.21),65.74 (16.55),0.026,0.056
gender = F (%),50.8%,47.8%,0.016,0.061
first_careunit = MICU (%),77.6%,79.5%,0.067,0.047
weight (mean (sd)),78.56 (23.58),83.17 (26.87),<0.001,0.182
saps (mean (sd)),19.63 (5.79),20.76 (5.44),<0.001,0.202
sofa (mean (sd)),5.31 (3.62),6.33 (3.79),<0.001,0.277
elix_score (mean (sd)),8.51 (7.45),10.07 (7.67),<0.001,0.207
vent = 1 (%),47.1%,58.3%,<0.001,0.226
vaso = 1 (%),27.1%,37.6%,<0.001,0.227


In [20]:
tab_ps_fmt <- tab_ps_df %>%
setNames(c("ctrl", "trtd", "pval", "smd")) %>%
mutate(cov = rownames(.)) %>%
mutate(ctrl = ifelse((grepl("%", cov) & grepl("=", cov)) | grepl("^\\s+", cov),
                     str_extract(ctrl, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     ctrl)) %>%
mutate(trtd = ifelse((grepl("%", cov) & grepl("=", cov)) | grepl("^\\s+", cov),
                     str_extract(trtd, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     trtd)) %>%
select(cov, ctrl, trtd, smd)

tab_ps_fmt

cov,ctrl,trtd,smd
n,1626,1626,
age (mean (sd)),66.58 (16.54),65.84 (17.07),0.044
gender = F (%),48.0%,49.0%,0.021
first_careunit = MICU (%),78.4%,78.9%,0.014
weight (mean (sd)),81.35 (24.69),79.50 (23.11),0.077
saps (mean (sd)),20.29 (5.42),19.80 (5.17),0.092
sofa (mean (sd)),5.99 (3.74),5.55 (3.50),0.123
elix_score (mean (sd)),9.54 (7.67),8.82 (7.52),0.096
vent = 1 (%),52.3%,49.6%,0.055
vaso = 1 (%),32.7%,31.2%,0.032


In [21]:
tab_wtd_fmt <- tab_wtd_df %>%
setNames(c("ctrl", "trtd", "pval")) %>%
mutate(cov = rownames(.)) %>%
mutate(ctrl = ifelse((grepl("%", cov) & grepl("=", cov)) | grepl("^\\s+", cov),
                     str_extract(ctrl, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     ctrl)) %>%
mutate(trtd = ifelse((grepl("%", cov) & grepl("=", cov)) | grepl("^\\s+", cov),
                     str_extract(trtd, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     trtd)) %>%
select(cov, ctrl, trtd, pval)

tab_wtd_fmt

cov,ctrl,trtd,pval
n,5739.48,5842.36,
age (mean (sd)),65.93 (16.29),65.15 (16.73),0.076
gender = F (%),49.2%,48.3%,0.523
first_careunit = MICU (%),77.9%,79.4%,0.178
weight (mean (sd)),82.31 (23.50),83.50 (25.25),0.163
saps (mean (sd)),23.13 (5.60),23.41 (5.18),0.134
sofa (mean (sd)),8.25 (4.25),8.52 (3.79),0.038
elix_score (mean (sd)),10.19 (7.45),10.02 (7.81),0.114
vent = 1 (%),50.7%,53.7%,0.025
vaso = 1 (%),30.7%,33.6%,0.020


In [22]:
tab_raw <-
tab_fmt %>%
full_join(tab_ps_fmt, by = "cov") %>%
mutate_all(funs(str_replace_all(., "\\s+$|^\\s+", ""))) %>%
mutate(cov = strsplit(cov, " ") %>% map_chr(first)) %>%
left_join(tab_ps_pval_df, by = "cov")

tab_raw

cov,ctrl.x,trtd.x,pval.x,smd.x,ctrl.y,trtd.y,smd.y,pval.y
n,3099,3262,,,1626,1626,,
age,66.69 (17.21),65.74 (16.55),0.026,0.056,66.58 (16.54),65.84 (17.07),0.044,0.209
gender,50.8%,47.8%,0.016,0.061,48.0%,49.0%,0.021,0.572
first_careunit,77.6%,79.5%,0.067,0.047,78.4%,78.9%,0.014,0.734
weight,78.56 (23.58),83.17 (26.87),<0.001,0.182,81.35 (24.69),79.50 (23.11),0.077,0.06
saps,19.63 (5.79),20.76 (5.44),<0.001,0.202,20.29 (5.42),19.80 (5.17),0.092,0.008
sofa,5.31 (3.62),6.33 (3.79),<0.001,0.277,5.99 (3.74),5.55 (3.50),0.123,0.001
elix_score,8.51 (7.45),10.07 (7.67),<0.001,0.207,9.54 (7.67),8.82 (7.52),0.096,0.005
vent,47.1%,58.3%,<0.001,0.226,52.3%,49.6%,0.055,0.108
vaso,27.1%,37.6%,<0.001,0.227,32.7%,31.2%,0.032,0.375


In [23]:
tab_fmt %>%
setNames(c("cov", "Non-Echo", "Echo", "p value", "SMD")) %>%
full_join(tab_wtd_fmt %>%
          setNames(c("cov", "Non-Echo (weighted cohort)", "Echo (weighted cohort)",
                     "p value (weighted cohort)")), by = "cov") %>%
full_join(tab_ps_fmt %>%
          setNames(c("cov", "Non-Echo (PS matching)", "Echo (PS matching)",
                     "SMD (PS matching)")), by = "cov") %>%
mutate_all(funs(str_replace_all(., "\\s+$|^\\s+", ""))) %>%
mutate(cov_ = strsplit(cov, " ") %>% map_chr(first)) %>%
left_join(tab_ps_pval_df %>%
          setNames(c("cov_", "p value (PS matching)")), by = "cov_")

cov,Non-Echo,Echo,p value,SMD,Non-Echo (weighted cohort),Echo (weighted cohort),p value (weighted cohort),Non-Echo (PS matching),Echo (PS matching),SMD (PS matching),cov_,p value (PS matching)
n,3099,3262,,,5739.48,5842.36,,1626,1626,,n,
age (mean (sd)),66.69 (17.21),65.74 (16.55),0.026,0.056,65.93 (16.29),65.15 (16.73),0.076,66.58 (16.54),65.84 (17.07),0.044,age,0.209
gender = F (%),50.8%,47.8%,0.016,0.061,49.2%,48.3%,0.523,48.0%,49.0%,0.021,gender,0.572
first_careunit = MICU (%),77.6%,79.5%,0.067,0.047,77.9%,79.4%,0.178,78.4%,78.9%,0.014,first_careunit,0.734
weight (mean (sd)),78.56 (23.58),83.17 (26.87),<0.001,0.182,82.31 (23.50),83.50 (25.25),0.163,81.35 (24.69),79.50 (23.11),0.077,weight,0.06
saps (mean (sd)),19.63 (5.79),20.76 (5.44),<0.001,0.202,23.13 (5.60),23.41 (5.18),0.134,20.29 (5.42),19.80 (5.17),0.092,saps,0.008
sofa (mean (sd)),5.31 (3.62),6.33 (3.79),<0.001,0.277,8.25 (4.25),8.52 (3.79),0.038,5.99 (3.74),5.55 (3.50),0.123,sofa,0.001
elix_score (mean (sd)),8.51 (7.45),10.07 (7.67),<0.001,0.207,10.19 (7.45),10.02 (7.81),0.114,9.54 (7.67),8.82 (7.52),0.096,elix_score,0.005
vent = 1 (%),47.1%,58.3%,<0.001,0.226,50.7%,53.7%,0.025,52.3%,49.6%,0.055,vent,0.108
vaso = 1 (%),27.1%,37.6%,<0.001,0.227,30.7%,33.6%,0.020,32.7%,31.2%,0.032,vaso,0.375


In [24]:
tab_full_fmt <-
tab_fmt %>%
setNames(c("cov", "Non-Echo", "Echo", "p value", "SMD")) %>%
full_join(tab_wtd_fmt %>%
          setNames(c("cov", "Non-Echo (weighted cohort)", "Echo (weighted cohort)",
                     "p value (weighted cohort)")), by = "cov") %>%
full_join(tab_ps_fmt %>%
          setNames(c("cov", "Non-Echo (PS matching)", "Echo (PS matching)",
                     "SMD (PS matching)")), by = "cov") %>%
mutate_all(funs(str_replace_all(., "\\s+$|^\\s+", ""))) %>%
mutate(cov_ = strsplit(cov, " ") %>% map_chr(first)) %>%
left_join(tab_ps_pval_df %>%
          setNames(c("cov_", "p value (PS matching)")), by = "cov_") %>%
left_join(missing %>%
          setNames(c("cov_", "Missing data")), by = "cov_") %>%
mutate(cov = str_replace_all(cov_, "_", " ")) %>%
mutate(cov = str_replace_all(cov, "lab|vs|flag|icd|first", "")) %>%
mutate_all(funs(str_replace_all(., "\\s+$|^\\s+", ""))) %>%
mutate(cov = case_when(
    grepl("icd|vs|lab|day", cov_) ~ toupper(cov),
    TRUE ~ tools::toTitleCase(cov)
)) %>%
mutate(cov = case_when(
    cov == "Gender" ~ "Gender (Female)",
    cov == "Careunit" ~ "Service Unit (MICU vs SICU)",
    cov == "Weight" ~ "Weight (Kg)",
    cov == "Elix Score" ~ "Elixhauser Score",
    cov == "Saps" ~ "SAPS Score",
    cov == "Sofa" ~ "SOFA Score",
    cov == "Vent" ~ "Mechanical Ventilation Use (1st 24 Hours)",
    cov == "Vaso" ~ "Vasopressor Use (1st 24 Hours)",
    cov == "Sedative" ~ "Sedative Use (1st 24 Hours)",
    cov == "ICU ADM WEEKDAY" ~ "Day of ICU Admission",
    cov == "Icu Adm Hour" ~ "Hour of ICU Admission",
    cov == "TEMP" ~ "Temperature (C)",
    cov == "HEART RATE" ~ "Heart Rate",
    cov %in% c("TROPONIN", "BNP", "CREATININE KINASE") ~ paste(cov, "(tested)"),
    TRUE ~ cov
)) %>%
select(-cov_) %>%
rename(Covariate = cov) %>%
select(1, 2, 3, 5, 4, 6, 7, 8, 9, 10, 11, 12, 13)

tab_full_fmt

Covariate,Non-Echo,Echo,SMD,p value,Non-Echo (weighted cohort),Echo (weighted cohort),p value (weighted cohort),Non-Echo (PS matching),Echo (PS matching),SMD (PS matching),p value (PS matching),Missing data
n,3099,3262,,,5739.48,5842.36,,1626,1626,,,NA
Age,66.69 (17.21),65.74 (16.55),0.056,0.026,65.93 (16.29),65.15 (16.73),0.076,66.58 (16.54),65.84 (17.07),0.044,0.209,0%
Gender (Female),50.8%,47.8%,0.061,0.016,49.2%,48.3%,0.523,48.0%,49.0%,0.021,0.572,0%
Service Unit (MICU vs SICU),77.6%,79.5%,0.047,0.067,77.9%,79.4%,0.178,78.4%,78.9%,0.014,0.734,0%
Weight (Kg),78.56 (23.58),83.17 (26.87),0.182,<0.001,82.31 (23.50),83.50 (25.25),0.163,81.35 (24.69),79.50 (23.11),0.077,0.06,9.4%
SAPS Score,19.63 (5.79),20.76 (5.44),0.202,<0.001,23.13 (5.60),23.41 (5.18),0.134,20.29 (5.42),19.80 (5.17),0.092,0.008,0%
SOFA Score,5.31 (3.62),6.33 (3.79),0.277,<0.001,8.25 (4.25),8.52 (3.79),0.038,5.99 (3.74),5.55 (3.50),0.123,0.001,0%
Elixhauser Score,8.51 (7.45),10.07 (7.67),0.207,<0.001,10.19 (7.45),10.02 (7.81),0.114,9.54 (7.67),8.82 (7.52),0.096,0.005,0%
Mechanical Ventilation Use (1st 24 Hours),47.1%,58.3%,0.226,<0.001,50.7%,53.7%,0.025,52.3%,49.6%,0.055,0.108,0%
Vasopressor Use (1st 24 Hours),27.1%,37.6%,0.227,<0.001,30.7%,33.6%,0.020,32.7%,31.2%,0.032,0.375,0%


In [25]:
tab_report <- tab_full_fmt %>%
filter(! Covariate %in% as.character(0:24)) %>%
select(1, 2, 3, 4, 9, 10, 11, 13)

tab_report

Covariate,Non-Echo,Echo,SMD,Non-Echo (PS matching),Echo (PS matching),SMD (PS matching),Missing data
n,3099,3262,,1626,1626,,NA
Age,66.69 (17.21),65.74 (16.55),0.056,66.58 (16.54),65.84 (17.07),0.044,0%
Gender (Female),50.8%,47.8%,0.061,48.0%,49.0%,0.021,0%
Service Unit (MICU vs SICU),77.6%,79.5%,0.047,78.4%,78.9%,0.014,0%
Weight (Kg),78.56 (23.58),83.17 (26.87),0.182,81.35 (24.69),79.50 (23.11),0.077,9.4%
SAPS Score,19.63 (5.79),20.76 (5.44),0.202,20.29 (5.42),19.80 (5.17),0.092,0%
SOFA Score,5.31 (3.62),6.33 (3.79),0.277,5.99 (3.74),5.55 (3.50),0.123,0%
Elixhauser Score,8.51 (7.45),10.07 (7.67),0.207,9.54 (7.67),8.82 (7.52),0.096,0%
Mechanical Ventilation Use (1st 24 Hours),47.1%,58.3%,0.226,52.3%,49.6%,0.055,0%
Vasopressor Use (1st 24 Hours),27.1%,37.6%,0.227,32.7%,31.2%,0.032,0%


In [26]:
data.table::fwrite(tab_full_fmt, file.path(data_dir, "tableone_full.csv"))
data.table::fwrite(tab_raw, file.path(data_dir, "tableone_raw.csv"))
data.table::fwrite(tab_report, file.path(data_dir, "tableone_report.csv"))
data.table::fwrite(wtd_pval, file.path(data_dir, "wtd_pval.csv"))